In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [2]:
file_list = os.listdir("/works/Data/Dacon/health_data_clf/")
file_list = [file for file in file_list if file.endswith(".csv")]
print(file_list)

['train_labels.csv', 'test_features.csv', 'train_features.csv', 'sample_submission.csv']


In [3]:
file_path = "/works/Data/Dacon/health_data_clf/"
train_label = pd.read_csv(file_path + file_list[0])
test_features = pd.read_csv(file_path + file_list[1])
train_features = pd.read_csv(file_path + file_list[2])
sample_submissuib = pd.read_csv(file_path + file_list[3])

print(train_label.shape)
print(test_features.shape)
print(train_features.shape)

(3125, 3)
(469200, 8)
(1875000, 8)


# Preprcessing

In [4]:
def return_index(arr, num):
    return arr.argsort()[-num:]

def prerpocessing_using_static(features_set, label_set):
    input_set = []
    target_set = []
    
    for id_idx in range(features_set.id.min(), features_set.id.max()+1):
        sample_features = features_set[["acc_x","acc_y","acc_z", "gy_x", "gy_y", "gy_z"]][features_set.id==id_idx].values
        
        #Fourier transform
        strength = np.fft.fft(sample_features, axis=0)
        strength = abs(strength)
        frequency = np.fft.fftfreq(len(sample_features), 1)
        strength_pos = strength[frequency>=0]
        frequency_pos = frequency[frequency>=0]
        freq_x = frequency_pos[return_index(strength_pos[:,0],3)].mean()
        freq_y = frequency_pos[return_index(strength_pos[:,1],3)].mean()
        freq_z = frequency_pos[return_index(strength_pos[:,2],3)].mean()
        freq_gy_x = frequency_pos[return_index(strength_pos[:,3],3)].mean()
        freq_gy_y = frequency_pos[return_index(strength_pos[:,4],3)].mean()
        freq_gy_z = frequency_pos[return_index(strength_pos[:,5],3)].mean()
        
        #Transform to static
        #Mean values
        mean_arr = sample_features.mean(axis=0)
        #Mag values
        mag_arr = abs(sample_features).mean(axis=0)
        #Std values
        std_arr = sample_features.std(axis=0)
        #Cov values
        cov_xy = np.cov(sample_features[:,0], sample_features[:,1])[0][1]
        cov_yz = np.cov(sample_features[:,1], sample_features[:,2])[0][1]
        cov_zx = np.cov(sample_features[:,2], sample_features[:,0])[0][1]
        cov_gy_xy = np.cov(sample_features[:,3], sample_features[:,4])[0][1]
        cov_gy_yz = np.cov(sample_features[:,4], sample_features[:,5])[0][1]
        cov_gy_zx = np.cov(sample_features[:,5], sample_features[:,3])[0][1]
        #Corr values
        cor_xy = cov_xy / (std_arr[0]*std_arr[1])
        cor_yz = cov_yz / (std_arr[1]*std_arr[2])
        cor_zx = cov_zx / (std_arr[2]*std_arr[0])
        cor_gy_xy = cov_gy_xy / (std_arr[3]*std_arr[4])
        cor_gy_yz = cov_gy_yz / (std_arr[4]*std_arr[5])
        cor_gy_zx = cov_gy_zx / (std_arr[5]*std_arr[3])
        #Start End point change of vector values
        start_point = sample_features[:3,:3].mean(axis=0)
        end_point = sample_features[-3:,:3].mean(axis=0)
        change_of_vector_st = np.cos((start_point * end_point).sum() / (np.sqrt(np.power(start_point, 2).sum()) * np.sqrt(np.power(end_point, 2).sum())))
        start_gy_point = sample_features[:3,3:].mean(axis=0)
        end_gy_point = sample_features[-3:,3:].mean(axis=0)
        change_of_vector_st_gy = np.cos((start_gy_point * end_gy_point).sum() / (np.sqrt(np.power(start_gy_point, 2).sum()) * np.sqrt(np.power(end_gy_point, 2).sum())))
        
        #Energy values
        shift_0_values = np.roll(sample_features, 1, axis=0)[1:]
        shift_1_values = sample_features[1:]
        energys = np.power((shift_0_values-shift_1_values),2)
        energy_strength = np.fft.fft(energys, axis=0)
        energy_strength = abs(energy_strength)
        energy_frequency = np.fft.fftfreq(len(energy_strength), 1)
        energy_strength_pos = energy_strength[energy_frequency>=0]
        energy_frequency_pos = energy_frequency[energy_frequency>=0]
        energy_freq_x = energy_frequency_pos[return_index(energy_strength_pos[:,0],3)].mean()
        energy_freq_y = energy_frequency_pos[return_index(energy_strength_pos[:,1],3)].mean()
        energy_freq_z = energy_frequency_pos[return_index(energy_strength_pos[:,2],3)].mean()
        energy_freq_gy_x = energy_frequency_pos[return_index(energy_strength_pos[:,3],3)].mean()
        energy_freq_gy_y = energy_frequency_pos[return_index(energy_strength_pos[:,4],3)].mean()
        energy_freq_gy_z = energy_frequency_pos[return_index(energy_strength_pos[:,5],3)].mean()
        energy_mean = energys.mean(axis=0)
        energy_std = energys.std(axis=0)
        #Max Min point num
        increase_shift_0 = np.where(shift_0_values-shift_1_values>=0, 1, -1)
        increase_shift_1 = np.roll(increase_shift_0, 1, axis=0)
        max_min_point_num = np.where(increase_shift_0[1:]*increase_shift_1[1:]==-1, 1, 0).sum(axis=0)
        
        #Make Set
        freqs = [freq_x, freq_y, freq_z, freq_gy_x, freq_gy_y, freq_gy_z,
                 energy_freq_x, energy_freq_y, energy_freq_z, energy_freq_gy_x, energy_freq_gy_y, energy_freq_gy_z]
        id_sample_set = list(mean_arr) + list(mag_arr) + list(std_arr)\
        + [cor_xy, cor_yz, cor_zx, cor_gy_xy, cor_gy_yz, cor_gy_zx]\
        + [change_of_vector_st, change_of_vector_st_gy]\
        + list(energy_mean) + list(energy_std) + list(max_min_point_num) + freqs

        #Append
        input_set.append(id_sample_set)
        target_set.append(label_set[label_set.id == id_idx].label.values[0])
    
    input_set = np.array(input_set)
    target_set = np.array(target_set)
    print("Input set : {} / Target set : {}".format(input_set.shape, target_set.shape))
    return input_set, target_set

In [5]:
input_set, target_set = prerpocessing_using_static(features_set=train_features, label_set=train_label)

Input set : (3125, 56) / Target set : (3125,)


# Correlation analysis

In [6]:
def correlation_ratio(categories, measurements):
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = np.sqrt(numerator/denominator)
    return eta

In [7]:
columns_list=["mean_x", "mean_y", "mean_z",
              "mean_gy_x", "mean_gy_y", "mean_gy_z",
              "mag_x", "mag_y", "mag_z",
              "mag_gy_x", "mag_gy_y", "mag_gy_z",
              "std_x", "std_y", "std_z",
              "std_gy_x", "std_gy_y", "std_gy_z",
              "cor_xy", "cor_yz", "cor_zx",
              "cor_gy_xy", "cor_gy_yz", "cor_gy_zx",
              "change_of_vector_st", "change_of_vector_st_gy",
              "energy_mean_x", "energy_mean_y", "energy_mean_z",
              "energy_mean_gy_x", "energy_mean_gy_y", "energy_mean_gy_z",
              "energy_std_x", "energy_std_y", "energy_std_z",
              "energy_std_gy_x", "energy_std_gy_y", "energy_std_gy_z",
              "max_min_point_num_x", "max_min_point_num_y", "max_min_point_num_z",
              "max_min_point_num_gy_x", "max_min_point_num_gy_y", "max_min_point_num_gy_z",
              "freq_x", "freq_y", "freq_z",
              "freq_gy_x", "freq_gy_y", "freq_gy_z",
              "energy_freq_x", "energy_freq_y", "energy_freq_z",
              "energy_freq_gy_x", "energy_freq_gy_y", "energy_freq_gy_z"]

etas = []
for feature_idx in range(input_set.shape[1]):
    measur_arr = input_set[:,feature_idx]
    cat_arr =  target_set
    eta = correlation_ratio(categories=target_set, measurements=measur_arr)
    etas.append(eta)
    
df = pd.DataFrame(np.array(etas).reshape(1,-1), columns=columns_list)

In [8]:
df.T

,0
mean_x,0.830833
mean_y,0.361217
mean_z,0.620176
mean_gy_x,0.430665
mean_gy_y,0.252139
mean_gy_z,0.345171
mag_x,0.749916
mag_y,0.696194
mag_z,0.674227
mag_gy_x,0.801971
